In [9]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_ascertain as dr_asc
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    # train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

# anxiety_label_type = "Anxiety"
anxiety_label_type = None
asc_label_type = dr_asc.SelfReports.AROUSAL

models = {
    # "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000),
    # "Bayes": GaussianNB(),
    "RF": RandomForestClassifier(n_estimators=50, max_features=3),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="error")
}

threshold = "fixed"
asc_threshold = "fixed"
test_size = 1.0

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr_a.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))


In [11]:
# TRAIN ON APD AND TEST ON ASCERTAIN
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_asc)
importlib.reload(dt)


print(f"APD PHASES {model_phases_apd} " + "-"*50)
x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
x_b, y_b = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold, normalize=True)
# drop subjects with noisy data
x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)]
y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)]
inds = pd.isnull(x_b).any(axis=1).to_numpy().nonzero()[0]
x_b = x_b.drop(labels=inds, axis=0)
y_b = y_b.drop(labels=inds, axis=0)

if anxiety_label_type is not None:
    x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True, get_importance=get_importance, drop_subject=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(e)
                # print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print("")
print("\n")

APD PHASES ['Baseline_Rest', 'BugBox_Relax', 'BugBox_Anticipate', 'BugBox_Exposure', 'BugBox_Break', 'Speech_Relax', 'Speech_Anticipate', 'Speech_Exposure', 'Speech_Break'] --------------------------------------------------
Ratio of negative to positive labels (0.1721419185282523) is under 0.333, oversampling negative class.
y_train:
0    232
1    173
Name: label, dtype: int64
y_test:
1    1522
0     506
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [ 951 1077]
Model RF, Predictions: [0 1], [ 166 1862]
Model XGB, Predictions: [0 1], [ 155 1873]

[('lf_hf_ratio', 1.720029303353318), ('mean_SCL', 1.3624000625007333), ('lf_rr', 1.257779419465423), ('hf_rr', 1.0907340588335879), ('bpm', 0.5003335813515074), ('sdnn', 0.3814115109009349), ('rmssd', -0.34513770282100475), ('SCR_rate', -0.995921596544698)]


[('mean_SCL', 0.1810588225981137), ('lf_rr', 0.13953898775367685), ('bpm', 0.13030593848382327), ('hf_rr', 0.1248697735818747), ('SCR_rate', 0.11320674186579316), ('rmssd', 0

In [12]:
# TRAIN ON ASCERTAIN AND TEST ON APD -- ALL
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_asc)
importlib.reload(dt)


x_a, y_a = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold, normalize=True)
# x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
# drop subjects with noisy data
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)

if anxiety_label_type is not None:
    x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True, get_importance=get_importance, drop_subject=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(e)
                # print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print("")
print("\n")

Ratio of negative to positive labels (0.1721419185282523) is under 0.333, oversampling negative class.
y_train:
1    1522
0     506
Name: label, dtype: int64
y_test:
0    232
1    173
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [ 86 319]
Model RF, Predictions: [0 1], [ 30 375]
Model XGB, Predictions: [0 1], [ 30 375]

[('mean_SCL', 0.5804997505418846), ('SCR_rate', 0.3473391348350961), ('lf_rr', 0.3461585735439906), ('hf_rr', -0.05873422360647623), ('bpm', -0.070126090942767), ('lf_hf_ratio', -0.7360409427257101), ('sdnn', -1.3742179506896846), ('rmssd', -1.599857426568144)]


[('mean_SCL', 0.1525683398029275), ('rmssd', 0.15130747806503164), ('sdnn', 0.13637733534774207), ('bpm', 0.13312557353541832), ('lf_hf_ratio', 0.11681528446092168), ('hf_rr', 0.11063250604642583), ('lf_rr', 0.10407823180120002), ('SCR_rate', 0.09509525094033297)]


[('sdnn', 0.15130739), ('mean_SCL', 0.14037783), ('lf_rr', 0.12932718), ('bpm', 0.118607596), ('lf_hf_ratio', 0.117781974), ('rmssd',

In [13]:
# TRAIN ON ASCERTAIN AND TEST ON APD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_asc)
importlib.reload(dt)


# for j, phases_apd in enumerate(model_phases_apd):
for j, phases_apd in enumerate([dr_a.Phases.BUG_EXPOSURE, dr_a.Phases.SPEECH_EXPOSURE]):
    print(f"APD PHASES {phases_apd} " + "-"*50)
    phases_apd = [phases_apd]
    x_a, y_a = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=asc_label_type, threshold=asc_threshold, normalize=True)
    x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
    # drop subjects with noisy data
    x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
    y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
    inds = pd.isnull(x_a).any(axis=1).to_numpy().nonzero()[0]
    x_a = x_a.drop(labels=inds, axis=0).reset_index(drop=True)
    y_a = y_a.drop(labels=inds, axis=0).reset_index(drop=True)

    if anxiety_label_type is not None:
        x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

    x_a = x_a.drop(["phaseId"], axis=1)
    x_b = x_b.drop(["phaseId"], axis=1)

    # make sure subjects from different datasets aren't labeled with the same index
    x_b["subject"] = x_b["subject"] + 500
    y_b["subject"] = y_b["subject"] + 500

    acc_results = {
        # "SVM": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    reports = {
        # "SVM": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    num_iters = 1
    get_importance = True
    for _ in range(num_iters):
        out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True, get_importance=get_importance, drop_subject=True)
        for model_name in acc_results:
            acc_results[model_name].append(out[model_name][0])
            reports[model_name].append(out[model_name][1])
            if get_importance:
                try:
                    print("")
                    feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                    feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                    print(feature_imp)
                except Exception as e:
                    print(e)
                    # print(out[model_name][0][2])
                print("")

    for model_name in acc_results.keys():
        print(f"Model evaluation metrics for {model_name}:")
        for i in range(len(reports[model_name])):
            report = reports[model_name][i]
            acc = acc_results[model_name][i]
            p = report["precision"]
            r = report["recall"]
            f1 = report["f1"]
            auc = report["auc"]
            print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
        print("")
    print("\n")

APD PHASES BugBox_Exposure --------------------------------------------------
Ratio of negative to positive labels (0.1721419185282523) is under 0.333, oversampling negative class.
Ratio of negative to positive labels (0.21621621621621623) is under 0.333, oversampling negative class.
y_train:
1    1522
0     506
Name: label, dtype: int64
y_test:
1    37
0    12
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [19 30]
Model RF, Predictions: [0 1], [ 3 46]
Model XGB, Predictions: [0 1], [ 4 45]

[('mean_SCL', 0.563381914609132), ('SCR_rate', 0.4334752479872908), ('lf_rr', 0.36958612296780896), ('hf_rr', -0.11800342541989668), ('bpm', -0.29996160800298727), ('lf_hf_ratio', -0.751363248311732), ('sdnn', -1.2162049041478786), ('rmssd', -1.5040646939411209)]


[('mean_SCL', 0.1467741187872302), ('rmssd', 0.14437724381972883), ('bpm', 0.13929633100707633), ('sdnn', 0.1358304391919295), ('lf_hf_ratio', 0.13161049187857268), ('lf_rr', 0.10838317623285186), ('hf_rr', 0.104228328449658